In [1]:
using Flux
using CUDA
using Flux:onehotbatch, onecold, crossentropy, throttle, @epochs

In [2]:
imgs = Flux.Data.MNIST.images()
labels = Flux.Data.MNIST.labels()

60000-element Array{Int64,1}:
 5
 0
 4
 1
 9
 2
 1
 3
 1
 4
 3
 5
 3
 ⋮
 7
 8
 9
 2
 9
 5
 1
 8
 3
 5
 6
 8

In [3]:
imgs[123]

In [4]:
labels[123]

2

In [5]:
X = hcat(float.(reshape.(imgs, :))...)
Y = onehotbatch(labels, 0:9)

10×60000 Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}:
 0  1  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  1  0  0  1  0  1  0  0  0  0     0  0  0  0  0  0  1  0  0  0  0  0
 0  0  0  0  0  1  0  0  0  0  0  0  0     0  0  0  1  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  1  0  0  1  0  1     0  0  0  0  0  0  0  0  1  0  0  0
 0  0  1  0  0  0  0  0  0  1  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0  0  1  0  …  0  0  0  0  0  1  0  0  0  1  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  1  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     1  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  1  0  0  0  0  0  1  0  0  0  1
 0  0  0  0  1  0  0  0  0  0  0  0  0     0  0  1  0  1  0  0  0  0  0  0  0

In [20]:
m = Chain(
Dense(28^2, 128, relu),
Dense(128, 64, relu),
Dense(64, 10),
softmax
)

Chain(Dense(784, 128, relu), Dense(128, 64, relu), Dense(64, 10), softmax)

In [21]:
loss(x,y) = crossentropy(m(x), y)
opt = ADAM()

ADAM(0.001, (0.9, 0.999), IdDict{Any,Any}())

In [22]:
accuracy(x, y) = mean(onecold(m(x)) .== onecold(y))

accuracy (generic function with 1 method)

In [23]:
dataset = Flux.Data.DataLoader(X, Y, batchsize=32)

Flux.Data.DataLoader{Tuple{Array{Float64,2},Flux.OneHotMatrix{Array{Flux.OneHotVector,1}}}}(([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], Bool[0 1 … 0 0; 0 0 … 0 0; … ; 0 0 … 0 1; 0 0 … 0 0]), 32, 60000, true, 60000, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  59991, 59992, 59993, 59994, 59995, 59996, 59997, 59998, 59999, 60000], false)

In [24]:
evalcb = () -> @show(loss(X, Y))

#7 (generic function with 1 method)

In [25]:
Flux.train!(loss, params(m), dataset, opt, cb=throttle(evalcb, 10))

loss(X, Y) = 2.3019726f0
loss(X, Y) = 0.187487f0
loss(X, Y) = 0.14902721f0
